## Import Credentials

In [0]:
%run "./Config"

In [0]:
# Mounting Directory for Yelp JSONs Intake
storageAccount = "gen10datafund2205"
storageContainer = "group1-marketing-analytics"
mount_point = "/mnt/group1/zipcodein"
    
    
configs2 = {"fs.azure.account.auth.type": "OAuth",
   "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
   "fs.azure.account.oauth2.client.id": clientid,
   "fs.azure.account.oauth2.client.secret": clientSecret,
   "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/d46b54b2-a652-420b-aa5a-2ef7f8fc706e/oauth2/token",
   "fs.azure.createRemoteFileSystemDuringInitialization": "true"}

try:
    dbutils.fs.unmount(mount_point)
except:
    pass


dbutils.fs.mount(
source = f"abfss://{storageContainer}@{storageAccount}.dfs.core.windows.net/", 
mount_point = mount_point, 
extra_configs = configs2)

/mnt/group1/zipcodein has been unmounted.
Out[1]: True

In [0]:
from pyspark.sql.functions import udf,col

In [0]:
income = spark.read.format('json').load(mount_point+'/CensusIncome_cleaned.json')

In [0]:
education = spark.read.format('json').load(mount_point+'/education_cleaned.json')

In [0]:
zips = spark.read.format('csv').options(header='True').load(mount_point+'/zip_code_database.csv')

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1390287949281470> in <module> 
 ----> 1 zips = spark . read . format ( 'csv' ) . options ( header = 'True' ) . load ( mount_point + '/business.csv' ) 

 NameError : name 'mount_point' is not defined

In [0]:
df = income.join(education,["ZipCode"])

In [0]:
zips = zips.select(['zip','state'])
zips = zips.withColumnRenamed('zip', 'ZipCode')
zips = zips.withColumnRenamed('state', 'StateAbbreviation')


In [0]:
df = df.join(zips, ["Zipcode"])

In [0]:
states = ['AZ','CA', 'DE', 'FL', 'ID', 'IL', 'IN', 'LA', 'MO', 'NV', 'NJ', 'PA', 'TN']
df = df.filter(df.StateAbbreviation.isin(states))

In [0]:
df = df.withColumnRenamed('CensusEducationPct', 'DegreePercentage')

In [0]:
df.write.format('json').mode('overwrite').save(
    mount_point+'/ZipCodeTable_cleaned.json')

In [0]:
# write to the Zip table
try:
    df.write.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
        .mode("append") \
        .option("dbtable", 'dbo.ZipCodeTable') \
        .option("user", user) \
        .option("password", password) \
        .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
        .save()
except:
    print('An Error Occured.')